# First Agent

In [1]:
# Agent.
#   'AI' 를 사용하여
#   주어진 '목적'을 수행하기 위해
#    주어진 '도구'를 AI 판단하에 사용하면서
#    여러'단계'를 거쳐 목적 수행

In [2]:
from dotenv import load_dotenv
load_dotenv()

from langchain_openai.chat_models.base import ChatOpenAI
llm = ChatOpenAI(temperature=0.1)

In [3]:
prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

In [4]:
llm.invoke(prompt)

AIMessage(content='= $4923.38', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 59, 'total_tokens': 65, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Cr0o8vuaL3ulDHUBfoyFFlenjcxS1', 'finish_reason': 'stop', 'logprobs': None}, id='run--b8e1a597-bacb-40af-8d14-43eb385e03dc-0', usage_metadata={'input_tokens': 59, 'output_tokens': 6, 'total_tokens': 65, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [5]:
# 수식의 결과는 5,273.38 이 나와야 하는데
# LLM 의 결과는 4923.38 이란다!?!?



In [6]:
# LLM 이 못하는 것
#  수식 계산.
#  실시간 정보
#   ...

In [7]:
# ★LLM 은 산술연산을 할수 없다!!
# LLM은 '산술연산'을 수행하지 않습니다.  이런 계산은 AI보다 '계산기'가 더 잘한다!
# LLM 은 text 를 생성해내는 겁니다. 문장 시퀀스의 다음 token 이 무엇인지 통계적으로 추측하는 겁니다.

# 이러한 LLM 의 오류를 잡기위해선 agent 를 제공해주어야 한다.
# 그리고 agent 를 위한 tool(들) 을 만들고, agent 가 tool 을 선택해서 실행하는 것이다.


# Agent 사용

In [8]:
from langchain.agents.initialize import initialize_agent

https://python.langchain.com/api_reference/langchain/agents/langchain.agents.initialize.initialize_agent.html#initialize-agent

```python
langchain.agents.initialize.initialize_agent(
    tools: Sequence[BaseTool],   # tools 들 전달
    llm: BaseLanguageModel,      # llm 전달
    agent: AgentType | None = None,   # AgentType 전달
    callback_manager: BaseCallbackManager | None = None,
    agent_path: str | None = None,
    agent_kwargs: dict | None = None,
    *,
    tags: Sequence[str] | None = None,
    **kwargs: Any,
)
```

In [9]:
from langchain.agents.agent_types import AgentType

In [10]:
# agent 에 전달할 tool 준비
# 간단한 tool 을 만들어 보자

# 함수 정의
def plus(a, b):
    return a + b

# 이 함수를 우리 agent 가 실제로 사용할수 있는 tool 로 변환해서 전달해주어야 한다
#   여기선 우린 StructuredTool 를 사용해보겠다.

In [11]:
from langchain_core.tools.structured import StructuredTool

# https://python.langchain.com/api_reference/core/tools/langchain_core.tools.structured.StructuredTool.html
# Tool that can operate on any number of inputs.
# Initialize the tool.

In [12]:
# agent 생성
agent = initialize_agent(
    tools = [  # tool(들) 전달
        StructuredTool.from_function(
            func=plus,   # 함수를 전달 -> tool변환
            name="Sum Calculator",   # tool 이름
            description="Use this to perform sums of two numbers. This tool take two arguments, both  should be numbers.",
        ),        
    ],
    llm = ChatOpenAI(temperature=0.1, model='gpt-4o'), 
    agent = AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # agent type
    verbose = True,   # agent 가 하는 작업의 모든 과정및 추론을 보여줌.
)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_944\1956366253.py:2: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [13]:
# StructuredTool => tool 인 plus(a, b) 가 '여러개의 input' 을 갖는 툴

In [14]:
# agent 호출
# prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"
agent.invoke(prompt)



> Entering new AgentExecutor chain...
To find the total cost, I will sum these numbers step by step.

First, I'll add the first two numbers: $355.39 and $924.87.

Action:
```
{
  "action": "Sum Calculator",
  "action_input": {"a": 355.39, "b": 924.87}
}
```
Observation: 1280.26
Thought:To continue summing the numbers, I'll add the next number, $721.2, to the current total, $1280.26.

Action:
```
{
  "action": "Sum Calculator",
  "action_input": {"a": 1280.26, "b": 721.2}
}
```

Observation: 2001.46
Thought:Action:
```
{
  "action": "Sum Calculator",
  "action_input": {"a": 2001.46, "b": 1940.29}
}
```
Observation: 3941.75
Thought:Action:
```
{
  "action": "Sum Calculator",
  "action_input": {"a": 3941.75, "b": 573.63}
}
```

Observation: 4515.38
Thought:Action:
```
{
  "action": "Sum Calculator",
  "action_input": {"a": 4515.38, "b": 65.72}
}
```
Observation: 4581.1
Thought:Action:
```
{
  "action": "Sum Calculator",
  "action_input": {"a": 4581.1, "b": 35.00}
}
```
Observation: 4616

{'input': 'Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12',
 'output': 'The total cost is $5273.38.'}

In [15]:
# 어떤 일이 일어나고 있던 걸까?
# - Langchain 은 우리의 tool 에 의해 생성된 prompt 를 만들어서 LLM 에게 요청
# - LLM 은 tool 을 위한 input 과 함께 응답.
# - LangChain 은 그 답변을 가져와서 (이때 OutputParser 사용) 우리 tool(함수) 를 우리 컴퓨터에서 실행한다. 
